# Jupiter notebook for the ninth seminar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import trange, tqdm
%config InlineBackend.figure_format = 'svg' 
from pylab import rcParams
rcParams['figure.figsize'] = 8,5
%matplotlib inline

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
print("Version of tensorflow:",tf.__version__)
import sys
from IPython.display import clear_output

Version of tensorflow: 2.7.0


In [ ]:
# Downloading dataset
data_train = tfds.load(name="fashion_mnist", split="train").prefetch(60000).cache()
data_test  = tfds.load(name="fashion_mnist", split="test" ).prefetch(10000).cache()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteBSSVAN/fashion_mnist-train.tfrecord


  0%|          | 0/60000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteBSSVAN/fashion_mnist-test.tfrecord


  0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


We need to create a complex model, the best way to do it is to use tf.keras.Model.

In [21]:
def unwrap(x):
  # Unwraps dataset to the appropriate form
  img = tf.reshape(tf.cast(x['image'], 'float32')/255., [28**2])
  lbl = x['label']
  return (img, lbl)

In [22]:
batch_size = 128
inputs = tf.keras.Input(shape=(28*28), name='input')
layer1UP = tf.keras.layers.Dense(64, activation='relu', name='1st_layer_UP')(inputs)
layer1DOWN = tf.keras.layers.Dense(32, activation='relu', name='1st_layer_DOWN')(inputs)
layer2UP = tf.keras.layers.Dense(32, activation='relu', name='2nd_layer_UP')(layer1UP)
layer2DOWN = tf.keras.layers.Dense(10, name='2nd_layer_DOWN')(layer1DOWN)
layer3UP = tf.keras.layers.Dense(10, name='3rd_layer_UP')(layer2UP)
average = tf.keras.layers.Average(name='outputs')([layer3UP, layer2DOWN])
#outputs = tf.keras.layers.Activation(activation='softmax')(average)

model = tf.keras.Model(inputs=inputs, outputs=average)

In [23]:
model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.metrics.SparseCategoricalAccuracy()])

In [24]:
num_epochs=10
model.fit(x=data_train.shuffle(60000).map(unwrap).batch(batch_size),
          epochs=num_epochs,
          validation_data=data_test.map(unwrap).batch(4096))

Epoch 1/10
469/469 [==============================] - 4s 7ms/step - loss: 0.5942 - sparse_categorical_accuracy: 0.8013 - val_loss: 0.4738 - val_sparse_categorical_accuracy: 0.8348
Epoch 2/10
469/469 [==============================] - 3s 7ms/step - loss: 0.4173 - sparse_categorical_accuracy: 0.8534 - val_loss: 0.4492 - val_sparse_categorical_accuracy: 0.8406
Epoch 3/10
469/469 [==============================] - 3s 7ms/step - loss: 0.3749 - sparse_categorical_accuracy: 0.8674 - val_loss: 0.4125 - val_sparse_categorical_accuracy: 0.8546
Epoch 4/10
469/469 [==============================] - 3s 7ms/step - loss: 0.3569 - sparse_categorical_accuracy: 0.8727 - val_loss: 0.3867 - val_sparse_categorical_accuracy: 0.8615
Epoch 5/10
469/469 [==============================] - 3s 7ms/step - loss: 0.3386 - sparse_categorical_accuracy: 0.8774 - val_loss: 0.3718 - val_sparse_categorical_accuracy: 0.8671
Epoch 6/10
469/469 [==============================] - 3s 7ms/step - loss: 0.3227 - sparse_categorica

In [25]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 784)]        0           []                               
                                                                                                  
 1st_layer_UP (Dense)           (None, 64)           50240       ['input[0][0]']                  
                                                                                                  
 2nd_layer_UP (Dense)           (None, 32)           2080        ['1st_layer_UP[0][0]']           
                                                                                                  
 1st_layer_DOWN (Dense)         (None, 32)           25120       ['input[0][0]']                  
                                                                                            